In [1]:
import csv
import json
import time
from kafka import KafkaProducer

print("Import 완료")

Import 완료


In [2]:
# Question 3: Test connection
producer = KafkaProducer(
    bootstrap_servers='localhost:9092',
    value_serializer=lambda v: json.dumps(v).encode('utf-8'),
    # Performance optimizations (without compression for faster initial connection)
    batch_size=16384,           # 16KB batch
    linger_ms=10,               # 10ms wait for batching
    acks=1                      # Don't wait for all replicas
)

# Check if connected (Question 3 answer)
connected = producer.bootstrap_connected()
print(f"Connection status: {connected}")

Connection status: True


In [3]:
# Question 4: Send taxi data
def main():
    csv_file = '../data/green_tripdata_2019-10.csv'  # Fixed path
    
    # Required columns only (as per homework)
    required_columns = [
        'lpep_pickup_datetime',
        'lpep_dropoff_datetime',
        'PULocationID',
        'DOLocationID',
        'passenger_count',
        'trip_distance',
        'tip_amount'
    ]
    
    t0 = time.time()  # Start time
    
    sent_count = 0
    
    with open(csv_file, 'r', newline='', encoding='utf-8') as file:
        reader = csv.DictReader(file)
        
        for row in reader:
            # Filter to required columns only
            filtered_row = {col: row[col] for col in required_columns if col in row}
            
            # Send to green-trips topic (async)
            producer.send('green-trips', value=filtered_row)
            
            sent_count += 1
            
            # Progress update every 50k messages
            if sent_count % 50000 == 0:
                print(f"Sent {sent_count} messages ({(sent_count/476387)*100:.1f}%)")
    
    print(f"Flushing {sent_count} messages...")
    # Ensure all messages are delivered
    producer.flush()
    
    t1 = time.time()  # End time
    took = t1 - t0
    
    print(f"\nTotal messages sent: {sent_count}")
    print(f"Time taken: {took:.2f} seconds")  # Question 4 answer
    print(f"Throughput: {sent_count/took:.0f} messages/sec")

if __name__ == "__main__":
    main()

Sent 50000 messages (10.5%)
Sent 100000 messages (21.0%)
Sent 150000 messages (31.5%)
Sent 200000 messages (42.0%)
Sent 250000 messages (52.5%)
Sent 300000 messages (63.0%)
Sent 350000 messages (73.5%)
Sent 400000 messages (84.0%)
Sent 450000 messages (94.5%)
Flushing 476386 messages...

Total messages sent: 476386
Time taken: 19.81 seconds
Throughput: 24051 messages/sec
